In [2]:
# off-the-shelf chain : 커스텀없이 일반적인 chain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit= 80,
    memory_key ="chat_history"
)

template ="""

    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = PromptTemplate.from_template(template),
    verbose=True #checking log of Prompt
)
chain.predict(question= "My name is Jungil")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.

    
    Human:My name is Jungil
    You:


> Finished chain.


'Hello Jungil! How can I assist you today?'

In [10]:
chain.predict(question= "I live in Dongtan")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.

    Human: My name is Jungil
AI: Hello Jungil! How can I assist you today?
    Human:I live in Dongtan
    You:


> Finished chain.


"That's great to know! How can I assist you with information or services related to Dongtan?"

In [12]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.

    Human: My name is Jungil
AI: Hello Jungil! How can I assist you today?
Human: I live in Dongtan
AI: That's great to know! How can I assist you with information or services related to Dongtan?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Jungil.'

In [13]:
#Chat based Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit = 120,
    memory_key = "chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human","{question}"),
])

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose = True,
)
chain.predict(question="My name is Jungil")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Jungil

> Finished chain.


'Nice to meet you, Jungil! How can I assist you today?'

In [1]:
chain.predict(question="I live in Seoul")

NameError: name 'chain' is not defined

In [15]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Jungil
AI: Nice to meet you, Jungil! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: what is my name?

> Finished chain.


'Your name is Jungil. How can I assist you further, Jungil?'

In [20]:
# LCEL (LangChain Expression Language) Based Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit = 120,
    memory_key = "chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human","{question}"),
])

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history= load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })   
    memory.save_context({"input": question},{"output": result.content}) 
    print(result)

invoke_chain("My name is jungil")

content='Hello Jungil! How can I assist you today?'


In [18]:
invoke_chain("What is my name?")

content='Your name is Jungil.'
